In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import statsmodels.formula.api as smf 
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split 
import itertools
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import make_scorer, r2_score
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

# Machine Learning Modeling

## LowPrice Small size models

### Model with Statistically significant variables

In [100]:
model_name = 'LowPrice_Smallsize'

In [2]:
df_ml = pd.read_csv('../../01_Outputs/modeling_2nd/df_model_'+model_name+'.csv').dropna()

In [3]:
target = 'TARGET_Weighted_RetailPrice'

In [4]:
X = df_ml.drop(target, axis=1)

In [5]:
y = df_ml[[target]]

In [6]:
y = np.log(y)

In [7]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [8]:
def rmse(y_true,y_pred):
    rmse = np.sqrt(mean_squared_error(y_true,y_pred))
    print('rmse',rmse)
    return rmse

In [9]:
def rsquare(y_true,y_pred):
    rsquare = r2_score(y_true, y_pred)
    print('rsquare',rsquare)
    return rsquare

In [149]:
def randomsearchRegression(model, parameters):
    from sklearn.model_selection import RandomizedSearchCV
    searcher = RandomizedSearchCV(model, parameters, cv=10, scoring=make_scorer(rsquare,greater_is_better=True))
    searcher.fit(train_X, train_y)
    print("Best CV params", searcher.best_params_)
    print("Best CV Rsquare", searcher.best_score_)
    print("Best train Rsquare", searcher.score(train_X, train_y))
    print("Test RMSE of best random search hypers:", searcher.score(test_X, test_y))
    return searcher.best_params_,  searcher.best_score_, searcher.score(test_X, test_y), searcher.score(train_X, train_y)

In [11]:
models_dict = {}
models_dict['random_forest'] = RandomForestRegressor()
models_dict['adaboost'] = AdaBoostRegressor()
models_dict['xgboost'] = xgb.XGBRegressor()

In [12]:
result_dict = {}
parameters_dct = {}
parameters_dct['random_forest'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    "max_depth": [i for i in range(3,11)],
                                    'max_features' : [i for i in range(1,train_X.shape[1])],
                                    "random_state": [1],
                                    "min_samples_leaf": [i for i in range(1, 21)]}
parameters_dct['adaboost'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1],
                                    'random_state': [1]}
parameters_dct['xgboost'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1],
                                    'max_depth':[1,3,5,7,10],
                                    'gamma': [0.01*i for i in range(0,1000,100)],
                                    'random_state': [1]}


In [13]:
model_list = ['random_forest', 'adaboost', 'xgboost']

In [14]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9293552504322073
rsquare 0.9180807680511375
rsquare 0.9365749749490798
rsquare 0.9089752252871253
rsquare 0.9287232200971527
rsquare 0.9074194681653875
rsquare 0.9311473199573728
rsquare 0.9165721324419487
rsquare 0.9038173317965283
rsquare 0.9206326715049596
rsquare 0.9425114604486973
rsquare 0.9297512607230931
rsquare 0.9490270780684547
rsquare 0.9413552026654732
rsquare 0.9500388774504261
rsquare 0.9328499262772579
rsquare 0.9519890591191957
rsquare 0.9308078225862474
rsquare 0.9112499156634687
rsquare 0.9446278532625338
rsquare 0.9054786746612642
rsquare 0.8834978144931571
rsquare 0.9050687455218335
rsquare 0.8746824347332031
rsquare 0.9008701256652992
rsquare 0.8794055499474612
rsquare 0.902220239304772
rsquare 0.8778786307193338
rsquare 0.8614482026061354
rsquare 0.8959188125471129
rsquare 0.919132647708897
rsquare 0.9030368887158744
rsquare 0.9234192230458212
rsquare 0.907708259130274
rsquare 0.9343859664829979
rsquare 0.9154403145938481
rsquare 0.9352187

#### Random Forest result

In [21]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 100, min_samples_leaf= 4, max_features= 64, max_depth= 8)

In [22]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=8, max_features=64, min_samples_leaf=4,
                      random_state=1)

In [23]:
pred_y = best_randomforest.predict(test_X)

In [24]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1395.0881346695846


1395.0881346695846

In [25]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9376906830027029

In [26]:
RF_importances = best_randomforest.feature_importances_

In [27]:
feature_names = list(X.columns)

In [28]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [29]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.165816
SPEC_published_kerb_weight_kg,0.153307
SPEC_maximum_torque_Nm,0.105713
SPEC_wheelrim_diameter_inch,0.060024
SPEC_FRDRseat_reclining_manual,0.051802
...,...
SPEC_luxurytrim_door_leather,0.000000
SPEC_folds_electrically,0.000000
SPEC_MirrorLink,0.000000
SPEC_Online_entertainment,0.000000


#### Adaboost result

In [30]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=100, learning_rate=1)


In [31]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=100, random_state=1)

In [32]:
pred_y = best_adaboost.predict(test_X)

In [33]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1868.2917764347735


1868.2917764347735

In [34]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8740406167084673

In [35]:
AB_importances = best_adaboost.feature_importances_

In [36]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [37]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,0.238833
SPEC_maximum_torque_Nm,0.215234
MSRP_prevMY_WeightedByVol,0.111470
SPEC_Seatmaterial_cloth,0.040783
SPEC_LED_lights,0.034269
...,...
PROFILE_Make_JEEP,0.000000
SPEC_RRseat_type_split_bench,0.000000
SPEC_memory,0.000000
SPEC_BelowseatStorage_passenger,0.000000


#### XGboost result

In [41]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=10,max_depth=7, learning_rate=1, gamma=0)

In [42]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [43]:
pred_y = best_xgboost.predict(test_X)

In [44]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1853.4247931516516


1853.4247931516516

In [45]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.912724310886108

In [46]:
XGB_importances = best_xgboost.feature_importances_

In [47]:
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=feature_names)

In [48]:
XGB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_FRDRseat_reclining_manual,0.362564
SPEC_actively_heated_cooled_battery,0.102917
SPEC_published_kerb_weight_kg,0.082386
SPEC_wheelrim_alloy,0.046211
SPEC_maximum_torque_Nm,0.045668
...,...
SPEC_configuration_horizontal,0.000000
SPEC_headrestraints_num,0.000000
SPEC_Console_other,0.000000
SPEC_includes_TR_ESP_adjustment,0.000000


In [49]:
RF_vars = list(RF_importances.sort_values(by='importances', ascending=False)[:200].index)

In [50]:
AB_vars = list(AB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [51]:
XGB_vars = list(XGB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [98]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [101]:
pd.DataFrame(featureimp_variables).to_csv('variables_union'+ model_name +'.csv', index=False)

In [95]:
len(featureimp_variables)

205

### Model with reduced variables Union of Intersections (205 variables)

In [54]:
X = df_ml.drop(target, axis=1)

In [55]:
X_featureimp = X[featureimp_variables]

In [56]:
y = df_ml[[target]]

In [57]:
y = np.log(y)

In [58]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [88]:
result_dict_2 = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9469395345229977
rsquare 0.936403000875249
rsquare 0.956510054583549
rsquare 0.9548458966252434
rsquare 0.9488158323156597
rsquare 0.9349009947421657
rsquare 0.9583512806644868
rsquare 0.9423531853892028
rsquare 0.9243016997677412
rsquare 0.9496056236297234
rsquare 0.9459593688742531
rsquare 0.9333846504797504
rsquare 0.9529576840478209
rsquare 0.9476033108066096
rsquare 0.9521829531662256
rsquare 0.9355267864010257
rsquare 0.9561874113187978
rsquare 0.9360497895532689
rsquare 0.9200592440396618
rsquare 0.9476656033032141
rsquare 0.9136556680546201
rsquare 0.9056392357077768
rsquare 0.9241343947565246
rsquare 0.9119593392297168
rsquare 0.9058334773842462
rsquare 0.88839594882276
rsquare 0.9197765548426194
rsquare 0.9086190272347037
rsquare 0.8929346276674308
rsquare 0.9048341422202143
rsquare 0.9327196298148388
rsquare 0.9121864806007818
rsquare 0.9343700874075807
rsquare 0.907543490242686
rsquare 0.9359363303257403
rsquare 0.9195831255556677
rsquare 0.941096427

#### Random Forest result

In [60]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 100, min_samples_leaf= 1, max_features= 55, max_depth= 9)

In [61]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=55, random_state=1)

In [62]:
pred_y = best_randomforest.predict(test_X)

In [63]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1213.6408649976984


1213.6408649976984

In [64]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9542656608541551

In [65]:
RF_importances = best_randomforest.feature_importances_

In [66]:
feature_names = list(X_featureimp.columns)

In [67]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [68]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,1.819446e-01
MSRP_prevMY_WeightedByVol,1.789520e-01
SPEC_maximum_torque_Nm,1.271290e-01
SPEC_FRDRseat_reclining_manual,5.873769e-02
Segment_ENTRY,4.585014e-02
...,...
SPEC_Emission_control_level,9.920326e-06
SPEC_emergency_driver_intervention_system,3.888368e-06
SPEC_rooftype_None,2.678553e-06
SPEC_crawl_control,1.481962e-06


#### Adaboost result

In [69]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=500, learning_rate=1)


In [70]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=500, random_state=1)

In [71]:
pred_y = best_adaboost.predict(test_X)

In [72]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1812.8070719061852


1812.8070719061852

In [73]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8785589227398382

In [74]:
AB_importances = best_adaboost.feature_importances_

In [75]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [76]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,0.225385
SPEC_maximum_torque_Nm,0.177923
MSRP_prevMY_WeightedByVol,0.100661
SPEC_Seatmaterial_cloth,0.038140
SPEC_LED_lights,0.030715
...,...
SPEC_rooftype_standard,0.000000
SPEC_FRDRseat_reclining_memory,0.000000
SPEC_configuration_V,0.000000
SPEC_Knee_airbags,0.000000


#### XGboost result

In [89]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=1000, max_depth=3, learning_rate=1, gamma=1.0)

In [90]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [91]:
pred_y = best_xgboost.predict(test_X)

In [92]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2739.1887245458006


2739.1887245458006

In [93]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8062818540252187

### Model with reduced variables of Intersections (99 variables)

In [102]:
featureimp_variables = list(set(RF_vars).intersection(set(XGB_vars)).intersection(set(AB_vars)))

In [103]:
len(featureimp_variables)

99

In [105]:
pd.DataFrame(featureimp_variables).to_csv('variables_intersections_'+model_name+'.csv', index=False)

In [106]:
X = df_ml.drop(target, axis=1)

In [107]:
X_featureimp = X[featureimp_variables]

In [108]:
y = df_ml[[target]]

In [109]:
y = np.log(y)

In [110]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [111]:
result_dict_3 = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9333453544707748
rsquare 0.920331910768384
rsquare 0.9410949191859446
rsquare 0.9367162279529111
rsquare 0.9317357047278408
rsquare 0.9222621427740156
rsquare 0.943473255898287
rsquare 0.9249270255251091
rsquare 0.9102840722858225
rsquare 0.9343554863171163
rsquare 0.9305691340490021
rsquare 0.9184721506063032
rsquare 0.9392118933666486
rsquare 0.9313355897735773
rsquare 0.9247540747197066
rsquare 0.9138696558833458
rsquare 0.9398913463579077
rsquare 0.9255772134311727
rsquare 0.9093057103683466
rsquare 0.9276247978247861
rsquare 0.9468171994188245
rsquare 0.9365684572469727
rsquare 0.9575656013846781
rsquare 0.954046729774031
rsquare 0.9492983583989543
rsquare 0.9373553697567532
rsquare 0.9579038730601708
rsquare 0.9416863752606267
rsquare 0.9265647715226952
rsquare 0.9492179082124095
rsquare 0.9161233426745932
rsquare 0.9047021589607375
rsquare 0.924012812133894
rsquare 0.9104803387716317
rsquare 0.9066062347642775
rsquare 0.8913209118689516
rsquare 0.91736259

#### Random Forest result

In [112]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 1000, min_samples_leaf= 6, max_features= 160, max_depth= 7)

In [113]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=7, max_features=160, min_samples_leaf=6,
                      n_estimators=1000, random_state=1)

In [114]:
pred_y = best_randomforest.predict(test_X)

In [115]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1555.7526828921777


1555.7526828921777

In [116]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9238220178125645

In [117]:
RF_importances = best_randomforest.feature_importances_

In [118]:
feature_names = list(X_featureimp.columns)

In [119]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [120]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,4.817074e-01
MSRP_prevMY_WeightedByVol,1.686413e-01
SPEC_maximum_torque_Nm,8.458342e-02
SPEC_FRDRseat_reclining_manual,6.154642e-02
SPEC_wheelrim_alloy,2.579668e-02
...,...
SPEC_IIHS_smalloverlapFRDR_Marginal,4.376902e-05
SPEC_RRseat_reclining_manual,2.733765e-05
PROFILE_MY_2023,1.837291e-05
SPEC_height_adjustment865,1.627500e-06


#### Adaboost result

In [121]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=500, learning_rate=1)


In [122]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=500, random_state=1)

In [123]:
pred_y = best_adaboost.predict(test_X)

In [124]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1814.2418599001905


1814.2418599001905

In [125]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8806618571637091

In [126]:
AB_importances = best_adaboost.feature_importances_

In [127]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [128]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,0.220508
SPEC_maximum_torque_Nm,0.193887
MSRP_prevMY_WeightedByVol,0.098155
SPEC_wheelrim_width_inch,0.041293
SPEC_Seatmaterial_cloth,0.040722
...,...
SPEC_configuration_V,0.000000
SPEC_programmable_distance,0.000000
SPEC_in_dash_CD,0.000000
SPEC_rooftype_standard,0.000000


#### XGboost result

In [134]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=100, max_depth=5, learning_rate=0.1, gamma=0)

In [135]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [136]:
pred_y = best_xgboost.predict(test_X)

In [137]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 933.4605882485242


933.4605882485242

In [138]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9751137063120483

## MidPrice_Largesize models

### Model with Statistically significant variables

In [139]:
model_name = 'MidPrice_Largesize'

In [140]:
df_ml = pd.read_csv('../../01_Outputs/modeling_2nd/df_model_'+model_name+'.csv').dropna()

In [141]:
target = 'TARGET_Weighted_RetailPrice'

In [142]:
X = df_ml.drop(target, axis=1)

In [143]:
y = df_ml[[target]]

In [144]:
y = np.log(y)

In [145]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [150]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.8438714614967406
rsquare 0.8179503714803709
rsquare 0.8183687689112238
rsquare 0.8278466982480295
rsquare 0.8011508331724854
rsquare 0.7967354813498743
rsquare 0.816545856209854
rsquare 0.8046946615272093
rsquare 0.8073942252257846
rsquare 0.7900418517867455
rsquare 0.7784587430985991
rsquare 0.7403821582948648
rsquare 0.7532346311191885
rsquare 0.7429473738431437
rsquare 0.7127369907111003
rsquare 0.709220031313329
rsquare 0.7335918060637632
rsquare 0.7205465186962057
rsquare 0.7059542939922475
rsquare 0.7265255714682586
rsquare 0.9014287310224707
rsquare 0.8863732014962118
rsquare 0.8941461364624462
rsquare 0.8959412967098432
rsquare 0.8820958353137736
rsquare 0.8758722302571701
rsquare 0.8837992955209706
rsquare 0.8854970432125578
rsquare 0.88218932347475
rsquare 0.8509831387787015
rsquare 0.9186132711674655
rsquare 0.9151681879449814
rsquare 0.911878980878971
rsquare 0.9102507674584565
rsquare 0.9034439711973711
rsquare 0.9013093015744001
rsquare 0.923705093

#### Random Forest result

In [151]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 1000, min_samples_leaf= 2, max_features= 100, max_depth= 9)

In [152]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=100, min_samples_leaf=2,
                      n_estimators=1000, random_state=1)

In [153]:
pred_y = best_randomforest.predict(test_X)

In [154]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2026.2631329861422


2026.2631329861422

In [155]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9376237679641425

In [156]:
RF_importances = best_randomforest.feature_importances_

In [157]:
feature_names = list(X.columns)

In [158]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [159]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_Autoairconditioner_zones,1.456682e-01
SPEC_Airconditioningtype_auto,1.455779e-01
SPEC_Airconditioningtype_manual,6.695405e-02
SPEC_LED_lowbeam,2.716684e-02
SPEC_FRPGseat_heated,2.513421e-02
...,...
SPEC_virtualassist_Google,2.773181e-06
SPEC_includes_steering_wheel_heating,1.210215e-06
SPEC_MP3_player,1.484162e-07
SPEC_Ground_view,0.000000e+00


#### Adaboost result

In [160]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=100, learning_rate=1)


In [161]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=100, random_state=1)

In [162]:
pred_y = best_adaboost.predict(test_X)

In [163]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3047.9036885865144


3047.9036885865144

In [164]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8277226324334617

In [165]:
AB_importances = best_adaboost.feature_importances_

In [166]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [167]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.084050
SPEC_Autoairconditioner_zones,0.068537
SPEC_Airconditioningtype_auto,0.061107
SPEC_overall_height_mm,0.060333
SPEC_subwoofer,0.052427
...,...
SPEC_motortorquelbft_max,0.000000
SPEC_RR_suspension_beam,0.000000
SPEC_AutomaticEmergencyBrake_lowspeed_front,0.000000
SPEC_touch_screen_controls,0.000000


#### XGboost result

In [168]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=500,max_depth=7, learning_rate=1, gamma=0)

In [169]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [170]:
pred_y = best_xgboost.predict(test_X)

In [171]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2268.558626276856


2268.558626276856

In [172]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9351162242932783

In [173]:
XGB_importances = best_xgboost.feature_importances_

In [174]:
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=feature_names)

In [175]:
XGB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_Airconditioningtype_auto,0.551901
SPEC_Anti_collision_system,0.315233
SPEC_parkingdistsys_FRRRSide,0.024976
SPEC_local_number_of_doors,0.017079
SPEC_Wheel_opening_moldings,0.015145
...,...
SPEC_configuration_V,0.000000
SPEC_luxurytrim_instrumentpanel_leather,0.000000
SPEC_Multi_collision_braking,0.000000
SPEC_Head_restraints,0.000000


In [176]:
RF_vars = list(RF_importances.sort_values(by='importances', ascending=False)[:200].index)

In [177]:
AB_vars = list(AB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [178]:
XGB_vars = list(XGB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [179]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [180]:
pd.DataFrame(featureimp_variables).to_csv('variables_union'+ model_name +'.csv', index=False)

In [181]:
len(featureimp_variables)

213

### Model with reduced variables Union of Intersections (213 variables)

In [182]:
X = df_ml.drop(target, axis=1)

In [183]:
X_featureimp = X[featureimp_variables]

In [184]:
y = df_ml[[target]]

In [185]:
y = np.log(y)

In [186]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [187]:
result_dict_2 = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9138317105503381
rsquare 0.9104369710339888
rsquare 0.9102056847036707
rsquare 0.9095488063411435
rsquare 0.8986645238774087
rsquare 0.8971019605135769
rsquare 0.9151085719148746
rsquare 0.9055637564660859
rsquare 0.9107090936839946
rsquare 0.8742844491943088
rsquare 0.9309306166527151
rsquare 0.921968955739559
rsquare 0.9193874986567723
rsquare 0.9165911034184461
rsquare 0.9143925807354912
rsquare 0.9117415988696321
rsquare 0.9306807740728366
rsquare 0.9179377964449388
rsquare 0.9162595376592152
rsquare 0.8851025255058127
rsquare 0.8478170273520691
rsquare 0.8200889685610192
rsquare 0.8173276330469254
rsquare 0.8277802536910669
rsquare 0.8086035247737582
rsquare 0.8001075444525996
rsquare 0.8182915273344902
rsquare 0.8035863580963868
rsquare 0.8047512533744021
rsquare 0.7899564114785949
rsquare 0.9550454789919949
rsquare 0.9482020482143914
rsquare 0.9502701887898193
rsquare 0.945569674007398
rsquare 0.9459104009703168
rsquare 0.9439950736129973
rsquare 0.958249

#### Random Forest result

In [188]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 1000, min_samples_leaf=4, max_features= 184, max_depth= 9)

In [189]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=184, min_samples_leaf=4,
                      n_estimators=1000, random_state=1)

In [190]:
pred_y = best_randomforest.predict(test_X)

In [191]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2093.7936280731888


2093.7936280731888

In [192]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9341567865603697

In [ ]:
RF_importances = best_randomforest.feature_importances_

In [ ]:
feature_names = list(X_featureimp.columns)

In [ ]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [ ]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,1.819446e-01
MSRP_prevMY_WeightedByVol,1.789520e-01
SPEC_maximum_torque_Nm,1.271290e-01
SPEC_FRDRseat_reclining_manual,5.873769e-02
Segment_ENTRY,4.585014e-02
...,...
SPEC_Emission_control_level,9.920326e-06
SPEC_emergency_driver_intervention_system,3.888368e-06
SPEC_rooftype_None,2.678553e-06
SPEC_crawl_control,1.481962e-06


#### Adaboost result

In [193]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=1000, learning_rate=1)


In [194]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=1000, random_state=1)

In [195]:
pred_y = best_adaboost.predict(test_X)

In [196]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2873.1047430442136


2873.1047430442136

In [197]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8498588651789698

In [ ]:
AB_importances = best_adaboost.feature_importances_

In [ ]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [ ]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,0.225385
SPEC_maximum_torque_Nm,0.177923
MSRP_prevMY_WeightedByVol,0.100661
SPEC_Seatmaterial_cloth,0.038140
SPEC_LED_lights,0.030715
...,...
SPEC_rooftype_standard,0.000000
SPEC_FRDRseat_reclining_memory,0.000000
SPEC_configuration_V,0.000000
SPEC_Knee_airbags,0.000000


#### XGboost result

In [198]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=500, max_depth=10, learning_rate=0.1, gamma=2.0)

In [199]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=2.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [200]:
pred_y = best_xgboost.predict(test_X)

In [201]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 4349.793696847234


4349.793696847234

In [202]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.47950174880771246

### Model with reduced variables of Intersections (128 variables)

In [203]:
featureimp_variables = list(set(RF_vars).intersection(set(XGB_vars)).intersection(set(AB_vars)))

In [204]:
len(featureimp_variables)

128

In [205]:
pd.DataFrame(featureimp_variables).to_csv('variables_intersections_'+model_name+'.csv', index=False)

In [206]:
X = df_ml.drop(target, axis=1)

In [207]:
X_featureimp = X[featureimp_variables]

In [208]:
y = df_ml[[target]]

In [209]:
y = np.log(y)

In [210]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [211]:
result_dict_3 = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9348196974523717
rsquare 0.9308772097522776
rsquare 0.9246552414016018
rsquare 0.9231134285350039
rsquare 0.9231982806238247
rsquare 0.9176412063931524
rsquare 0.9383388128857262
rsquare 0.9240940216231124
rsquare 0.927945744465775
rsquare 0.8920564358539841
rsquare 0.940219874123268
rsquare 0.9335428399884964
rsquare 0.9352215983176834
rsquare 0.9336953506352236
rsquare 0.9302520263691731
rsquare 0.9249179902244505
rsquare 0.9375761537081828
rsquare 0.9327954326284889
rsquare 0.9297312997942261
rsquare 0.8930436747972192
rsquare 0.7791381376025025
rsquare 0.7460016936111133
rsquare 0.7602858583326211
rsquare 0.7644588461857234
rsquare 0.7563703193748939
rsquare 0.7399384591336973
rsquare 0.7450827360689829
rsquare 0.7336399681586935
rsquare 0.7326977962448131
rsquare 0.7283761543231186
rsquare 0.936011892459667
rsquare 0.9281696843993549
rsquare 0.9232803559441258
rsquare 0.9242376315021023
rsquare 0.9134211196343277
rsquare 0.9184110942268634
rsquare 0.9327674

#### Random Forest result

In [212]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 300, min_samples_leaf= 8, max_features= 26, max_depth= 10)

In [213]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=10, max_features=26, min_samples_leaf=8,
                      n_estimators=300, random_state=1)

In [214]:
pred_y = best_randomforest.predict(test_X)

In [215]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2443.782426067423


2443.782426067423

In [216]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9047212553030742

In [217]:
RF_importances = best_randomforest.feature_importances_

In [218]:
feature_names = list(X_featureimp.columns)

In [219]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [220]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_Airconditioningtype_auto,0.124265
SPEC_LED_lowbeam,0.079095
SPEC_FRPGseat_heated,0.048915
SPEC_subwoofer,0.046460
SPEC_local_number_of_doors,0.036268
...,...
SPEC_12v_power_outlet,0.000038
SPEC_Electronic_traction_control,0.000032
SPEC_Front_airbag,0.000030
SPEC_gearchange_paddles,0.000009


#### Adaboost result

In [221]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=500, learning_rate=1)


In [222]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=500, random_state=1)

In [223]:
pred_y = best_adaboost.predict(test_X)

In [224]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2878.6656245816016


2878.6656245816016

In [225]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8482909391988239

In [226]:
AB_importances = best_adaboost.feature_importances_

In [227]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [228]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_Airconditioningtype_auto,0.098101
MSRP_prevMY_WeightedByVol,0.075583
SPEC_overall_height_mm,0.068973
SPEC_local_number_of_doors,0.065946
SPEC_subwoofer,0.056360
...,...
SPEC_remote_lock_unlock,0.000000
SPEC_Load_restraint,0.000000
SPEC_Alternate_suspension,0.000000
SPEC_auto_dimming,0.000000


#### XGboost result

In [229]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=100, max_depth=5, learning_rate=0.1, gamma=0)

In [230]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [231]:
pred_y = best_xgboost.predict(test_X)

In [232]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1587.381805150787


1587.381805150787

In [233]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9658049468433322

## MidPrice_Midsize models

### Model with Statistically significant variables

In [234]:
model_name = 'MidPrice_Midsize'

In [235]:
df_ml = pd.read_csv('../../01_Outputs/modeling_2nd/df_model_'+model_name+'.csv').dropna()

In [236]:
target = 'TARGET_Weighted_RetailPrice'

In [237]:
X = df_ml.drop(target, axis=1)

In [238]:
y = df_ml[[target]]

In [239]:
y = np.log(y)

In [240]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [241]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9319502216341343
rsquare 0.9246972182215212
rsquare 0.9247055867187148
rsquare 0.9312540475443628
rsquare 0.9249107809252943
rsquare 0.9096379581417415
rsquare 0.923681388940431
rsquare 0.9202535826665218
rsquare 0.9041252136565392
rsquare 0.9177660628005329
rsquare 0.9077716701754313
rsquare 0.9066359699932286
rsquare 0.903863422532428
rsquare 0.9067185501263003
rsquare 0.9036448949463453
rsquare 0.8879427357653152
rsquare 0.9043024755166394
rsquare 0.8875327027826176
rsquare 0.8832787804403416
rsquare 0.8928781556400472
rsquare 0.8679803630261518
rsquare 0.8560209921069086
rsquare 0.8694983839876591
rsquare 0.8631274875082598
rsquare 0.8576265184518032
rsquare 0.8498385497999992
rsquare 0.8546890521236713
rsquare 0.8544552571941719
rsquare 0.8388059505505573
rsquare 0.8540857715452834
rsquare 0.9532268635577605
rsquare 0.9527440227953808
rsquare 0.9489812009780727
rsquare 0.9521373238380605
rsquare 0.9520868312966537
rsquare 0.9368108469816221
rsquare 0.948757

#### Random Forest result

In [242]:
# best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 1000, min_samples_leaf= 2, max_features= 100, max_depth= 9)
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [243]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=8, max_features=143, n_estimators=1000,
                      random_state=1)

In [244]:
pred_y = best_randomforest.predict(test_X)

In [245]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2735.6289860109046


2735.6289860109046

In [246]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9443971779788827

In [247]:
RF_importances = best_randomforest.feature_importances_

In [248]:
feature_names = list(X.columns)

In [249]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [250]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,3.025205e-01
SPEC_maximum_torque_Nm,1.575391e-01
SPEC_rain_sensor,1.158070e-01
SPEC_doormirror_autodimming,6.633047e-02
SPEC_FRPGseat_reach_manual,5.578661e-02
...,...
SPEC_FRseatbelt_Center,6.909029e-07
SPEC_Mudflaps_FR,3.287054e-07
SPEC_includes_augmented_reality,8.925636e-08
SPEC_Side_airbag,1.626775e-08


#### Adaboost result

In [251]:
# best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=100, learning_rate=1)
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [252]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=500, random_state=1)

In [253]:
pred_y = best_adaboost.predict(test_X)

In [254]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3413.514492225525


3413.514492225525

In [255]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9114678264092403

In [256]:
AB_importances = best_adaboost.feature_importances_

In [257]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [258]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_maximum_torque_Nm,0.326566
MSRP_prevMY_WeightedByVol,0.216973
SPEC_wheelrim_width_inch,0.117433
SPEC_rain_sensor,0.050637
SPEC_doormirror_autodimming,0.048729
...,...
SPEC_engine_heater,0.000000
SPEC_Convertibleroof_glassRRwindow,0.000000
SPEC_FRPGseat_folding,0.000000
SPEC_towingclass_IV,0.000000


#### XGboost result

In [259]:
# best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=500,max_depth=7, learning_rate=1, gamma=0)
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [260]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [261]:
pred_y = best_xgboost.predict(test_X)

In [262]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2785.285448084341


2785.285448084341

In [263]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9489091952895724

In [264]:
XGB_importances = best_xgboost.feature_importances_

In [265]:
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=feature_names)

In [266]:
XGB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_Steering_plastic,0.158943
SPEC_maximum_torque_Nm,0.135754
SPEC_screen_size_in,0.124438
SPEC_Headlight_cleaners,0.069801
MSRP_prevMY_WeightedByVol,0.057183
...,...
SPEC_Charging_information,0.000000
SPEC_Electric_motor,0.000000
SPEC_instantaneous_fuel_consumption,0.000000
SPEC_Storage_box,0.000000


In [267]:
RF_vars = list(RF_importances.sort_values(by='importances', ascending=False)[:200].index)

In [268]:
AB_vars = list(AB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [269]:
XGB_vars = list(XGB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [270]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [271]:
pd.DataFrame(featureimp_variables).to_csv('variables_union'+ model_name +'.csv', index=False)

In [272]:
len(featureimp_variables)

203

### Model with reduced variables Union of Intersections (203 variables)

In [273]:
X = df_ml.drop(target, axis=1)

In [274]:
X_featureimp = X[featureimp_variables]

In [275]:
y = df_ml[[target]]

In [276]:
y = np.log(y)

In [277]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [278]:
result_dict_2 = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9308354959842812
rsquare 0.9201635269765
rsquare 0.9199034370044293
rsquare 0.9241341331118672
rsquare 0.9222317922100569
rsquare 0.9074330162007604
rsquare 0.9184495128073874
rsquare 0.9132660469084568
rsquare 0.8889125190357279
rsquare 0.9112571597438083
rsquare 0.9637610093110398
rsquare 0.9645322272128686
rsquare 0.9619603479907054
rsquare 0.9642847911562095
rsquare 0.9632347952164677
rsquare 0.9515952107722977
rsquare 0.9596383478779069
rsquare 0.9532190644899038
rsquare 0.9560121087481629
rsquare 0.9559215020714474
rsquare 0.9657082682929055
rsquare 0.9668461009028784
rsquare 0.9663458412539924
rsquare 0.9639244873284114
rsquare 0.9602752543772712
rsquare 0.9540065604340137
rsquare 0.9608463724684385
rsquare 0.9607431193795997
rsquare 0.9503517999456941
rsquare 0.9579983777049077
rsquare 0.8592856115379022
rsquare 0.8233752479683462
rsquare 0.8193377787723275
rsquare 0.8445010693145144
rsquare 0.8440106782169366
rsquare 0.8298573273514147
rsquare 0.8351415

#### Random Forest result

In [282]:
result_dict_2

{}

In [283]:
best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 100, min_samples_leaf=7, max_features= 328, max_depth= 9)
# best_randomforest = RandomForestRegressor(**result_dict_2['random_forest_param'])

In [284]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=328, min_samples_leaf=7,
                      random_state=1)

In [285]:
pred_y = best_randomforest.predict(test_X)

In [286]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2915.580361255957


2915.580361255957

In [287]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9382879750239846

In [288]:
RF_importances = best_randomforest.feature_importances_

In [289]:
feature_names = list(X_featureimp.columns)

In [290]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [291]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.528051
SPEC_maximum_torque_Nm,0.199531
SPEC_FRPGseat_reach_manual,0.109335
SPEC_Steering_plastic,0.014743
SPEC_wheelrim_width_inch,0.009937
...,...
SPEC_luxurytrim_instrumentpanel_suede,0.000000
SPEC_remote_air_conditioning_control,0.000000
SPEC_Charges,0.000000
SPEC_luxurytrim_centerconsole_suede,0.000000


#### Adaboost result

In [292]:
best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=1000, learning_rate=1)
# best_adaboost= AdaBoostRegressor(**result_dict_2['adaboost_param'])


In [293]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=1000, random_state=1)

In [294]:
pred_y = best_adaboost.predict(test_X)

In [295]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3359.025785689877


3359.025785689877

In [296]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9145764109510816

In [297]:
AB_importances = best_adaboost.feature_importances_

In [298]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [299]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_maximum_torque_Nm,0.326744
MSRP_prevMY_WeightedByVol,0.208722
SPEC_wheelrim_width_inch,0.118713
SPEC_rain_sensor,0.051773
SPEC_doormirror_autodimming,0.050974
...,...
SPEC_Tachometer,0.000000
SPEC_luxurytrim_centerconsole_gloss,0.000000
SPEC_configuration_V,0.000000
SPEC_Differential_lock,0.000000


#### XGboost result

In [300]:
best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=1000, max_depth=3, learning_rate=0.1, gamma=0.0)
# best_xgboost = xgb.XGBRegressor(**result_dict_2['xgboost_param'])

In [301]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [302]:
pred_y = best_xgboost.predict(test_X)

In [303]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2042.8702972557899


2042.8702972557899

In [304]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9715187576450373

### Model with reduced variables of Intersections (109 variables)

In [305]:
featureimp_variables = list(set(RF_vars).intersection(set(XGB_vars)).intersection(set(AB_vars)))

In [306]:
len(featureimp_variables)

109

In [307]:
pd.DataFrame(featureimp_variables).to_csv('variables_intersections_'+model_name+'.csv', index=False)

In [308]:
X = df_ml.drop(target, axis=1)

In [309]:
X_featureimp = X[featureimp_variables]

In [310]:
y = df_ml[[target]]

In [311]:
y = np.log(y)

In [312]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [313]:
result_dict_3 = {}
for model in model_list:
    print(model)
    result_dict_3[model + '_param'], \
    result_dict_3[model + '_cvacc'], \
    result_dict_3[model + '_testacc'],\
    result_dict_3[model + '_trainacc'] \
    = randomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9573679493397602
rsquare 0.9530318111758034
rsquare 0.9561363077640179
rsquare 0.954421740054278
rsquare 0.9493070792508518
rsquare 0.9426181645854552
rsquare 0.9488527458725826
rsquare 0.9515924814350262
rsquare 0.9262424135558188
rsquare 0.9447946779919868
rsquare 0.9431117176709859
rsquare 0.9356308864377189
rsquare 0.9390215696446282
rsquare 0.9401094359267453
rsquare 0.9358293936634692
rsquare 0.9239212678433095
rsquare 0.9340473041262596
rsquare 0.9331702267351469
rsquare 0.9072668673613585
rsquare 0.9267821957326141
rsquare 0.9483455067014428
rsquare 0.9412829157866107
rsquare 0.9475319495703973
rsquare 0.945758823944639
rsquare 0.9413835222952609
rsquare 0.9316742579917959
rsquare 0.9383801641323531
rsquare 0.9420726167414077
rsquare 0.9153903353553097
rsquare 0.933232163358506
rsquare 0.8773842934424833
rsquare 0.8494488857964756
rsquare 0.8551477188206915
rsquare 0.8653785230556997
rsquare 0.8596555062429384
rsquare 0.8501381976328223
rsquare 0.8558424

#### Random Forest result

In [314]:
# best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 300, min_samples_leaf= 8, max_features= 26, max_depth= 10)
best_randomforest = RandomForestRegressor(**result_dict_3['random_forest_param'])

In [315]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=124, min_samples_leaf=15,
                      random_state=1)

In [316]:
pred_y = best_randomforest.predict(test_X)

In [317]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3323.4505241282554


3323.4505241282554

In [318]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9160285423833608

In [319]:
RF_importances = best_randomforest.feature_importances_

In [320]:
feature_names = list(X_featureimp.columns)

In [321]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [322]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.539498
SPEC_maximum_torque_Nm,0.203162
SPEC_FRPGseat_reach_manual,0.111875
SPEC_wheelrim_diameter_inch,0.019835
SPEC_Steering_plastic,0.015223
...,...
SPEC_Ceramic_brakes,0.000000
SPEC_Charges,0.000000
SPEC_parkingassist_auto,0.000000
SPEC_Luxury_inner_roof_upholstery,0.000000


#### Adaboost result

In [325]:
# best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=500, learning_rate=1)
best_adaboost= AdaBoostRegressor(**result_dict_3['adaboost_param'])

In [326]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=0.1, n_estimators=1000, random_state=1)

In [327]:
pred_y = best_adaboost.predict(test_X)

In [328]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3548.8820708973


3548.8820708973

In [329]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9042696969384765

In [330]:
AB_importances = best_adaboost.feature_importances_

In [331]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [332]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_maximum_torque_Nm,0.348195
MSRP_prevMY_WeightedByVol,0.214599
SPEC_wheelrim_width_inch,0.117130
SPEC_rain_sensor,0.049198
SPEC_doormirror_autodimming,0.046044
...,...
PROFILE_Make_HYUNDAI,0.000000
SPEC_suspentionleveling_rear,0.000000
SPEC_in_dash_CD,0.000000
SPEC_average_speed,0.000000


#### XGboost result

In [336]:
# best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=100, max_depth=5, learning_rate=0.1, gamma=0)
best_xgboost = xgb.XGBRegressor(**result_dict_3['xgboost_param'])

In [337]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [338]:
pred_y = best_xgboost.predict(test_X)

In [339]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2153.882330082875


2153.882330082875

In [340]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9680535419476987